In [4]:
import streamlit as st
import pandas as pd
from datetime import datetime, time, timedelta

file_map = {
    ("Batu Caves - Pulau Sebang", "Weekdays"): [
        "timetables/batu_caves_weekdays_route_1.parquet",
        "timetables/batu_caves_weekdays_route_2.parquet",
    ],
    ("Batu Caves - Pulau Sebang", "Weekends"): [
        "timetables/batu_caves_weekends_route_1.parquet",
        "timetables/batu_caves_weekends_route_2.parquet",
    ],
    ("Tanjung Malim - Pelabuhan Klang", "Weekdays"): [
        "timetables/klang_weekdays_route_1.parquet",
        "timetables/klang_weekdays_route_2.parquet",
    ],
    ("Tanjung Malim - Pelabuhan Klang", "Weekends"): [
        "timetables/klang_weekends_route_1.parquet",
        "timetables/klang_weekends_route_2.parquet",
    ],
    ("Padang Besar - Butterworth", "Not applicable"): [
        "timetables/utara_padangbesar_1.parquet",
        "timetables/utara_padangbesar_2.parquet",
    ],
    ("Ipoh - Butterworth", "Not applicable"): [
        "timetables/utara_ipoh_1.parquet",
        "timetables/utara_ipoh_2.parquet",
    ],
}


selected_route = "Batu Caves - Pulau Sebang"
selected_schedule = "Weekdays"
departure = "KUALA LUMPUR"
destination = "PULAU SEBANG"

key = (selected_route, selected_schedule)
print(key)

# from above key get the both file in file_map
if key in file_map:
    files = file_map[key]
    print("Total files:", len(files))
    if len(files) == 2:    
        df1 = pd.read_parquet(files[0])
        df2 = pd.read_parquet(files[1])


print("Size of df1 shape", df1.shape)
print("Size of df2 shape", df2.shape)




# Function to find index of departure station in a DataFrame
def find_station_index(df, station):
    if 'STATION' not in df.columns:
        return float('inf')  # Not available
    matching_rows = df.index[df['STATION'] == station].tolist()
    return matching_rows[0] if matching_rows else float('inf')

# Get index of departure station in both DataFrames
idx1 = find_station_index(df1, departure)
idx2 = find_station_index(df2, departure)

# Debug: Show results
print(f"Index in df1: {idx1}")
print(f"Index in df2: {idx2}")

# Decide which DataFrame to use
if idx1 == float('inf') and idx2 == float('inf'):
    print("Departure station not found in either DataFrame.")
    chosen_df = None
elif idx1 <= idx2:
    print(f"Using df1 because {departure} appears earlier (or only in df1).")
    chosen_df = df1
else:
    print(f"Using df2 because {departure} appears earlier (or only in df2).")
    chosen_df = df2

# Step 1: Ensure chosen_df is available
if chosen_df is None:
    print("No valid DataFrame selected.")
    schedules = None
else:
    # Step 2: Check if both stations exist
    if departure not in chosen_df['STATION'].values:
        print(f"❌ Departure station '{departure}' not found.")
        schedules = pd.DataFrame()
    elif destination not in chosen_df['STATION'].values:
        print(f"❌ Destination station '{destination}' not found.")
        schedules = pd.DataFrame()
    else:
        valid_services = []

        for col in chosen_df.columns[1:]:  # Skip 'STATION'
            # Extract the specific row for departure and destination
            dep_row = chosen_df[chosen_df['STATION'] == departure]
            dest_row = chosen_df[chosen_df['STATION'] == destination]

            # Get values
            dep_time = dep_row.iloc[0][col]
            dest_time = dest_row.iloc[0][col]

            # 🔴 Enhanced: Check if dep_time is truly non-empty
            def is_non_empty_time(val):
                if pd.isna(val):
                    return False
                if isinstance(val, str):
                    return val.strip() != ''
                return True  # Treat non-string non-null as valid

            if not is_non_empty_time(dep_time):
                continue  # ❌ Skip: no departure time (empty, null, or whitespace)

            if not is_non_empty_time(dest_time):
                continue  # ❌ Skip: no arrival time

            # Optional: Time order validation (if both are strings like '6:16', '10:26')
            try:
                # Normalize time strings for comparison: convert HH:MM → minutes since midnight
                def time_to_minutes(t):
                    t_str = str(t).strip()
                    if ':' in t_str:
                        h, m = t_str.split(':')
                    elif len(t_str) <= 3:
                        h, m = "0", t_str.zfill(2)
                    else:
                        h, m = t_str[:-2], t_str[-2:]
                    return int(h) * 60 + int(m)

                dep_minutes = time_to_minutes(dep_time)
                dest_minutes = time_to_minutes(dest_time)

                if dep_minutes >= dest_minutes:
                    continue  # ❌ Skip: departure after or same as arrival
            except Exception:
                pass  # If parsing fails, assume order is OK

            # ✅ All checks passed
            valid_services.append({
                'Service_ID': col,
                'Departure_Station': departure,
                'Departure_Time': str(dep_time).strip(),  # Clean up whitespace
                'Arrival_Station': destination,
                'Arrival_Time': str(dest_time).strip()
            })

        # Final output
        if valid_services:
            schedules = pd.DataFrame(valid_services)
            print(f"✅ Found {len(schedules)} valid train service(s) from '{departure}' to '{destination}':")
            print(schedules)
        else:
            print(f"🚫 No valid train services found from '{departure}' to '{destination}'.")
            print("💡 Either no departure time, no arrival time, or invalid time sequence.")
            schedules = pd.DataFrame()

('Batu Caves - Pulau Sebang', 'Weekdays')
Total files: 2
Size of df1 shape (27, 24)
Size of df2 shape (27, 23)
Index in df1: 7
Index in df2: 19
Using df1 because KUALA LUMPUR appears earlier (or only in df1).
✅ Found 13 valid train service(s) from 'KUALA LUMPUR' to 'PULAU SEBANG':
   Service_ID Departure_Station Departure_Time Arrival_Station Arrival_Time
0        2007      KUALA LUMPUR           6:16    PULAU SEBANG         9:13
1        2011      KUALA LUMPUR           7:46    PULAU SEBANG        10:13
2        2015      KUALA LUMPUR           8:46    PULAU SEBANG        11:53
3        2019      KUALA LUMPUR          10:26    PULAU SEBANG        12:53
4        2027      KUALA LUMPUR          12:51    PULAU SEBANG        15:18
5        2031      KUALA LUMPUR          13:51    PULAU SEBANG        16:18
6        2035      KUALA LUMPUR          14:51    PULAU SEBANG        17:18
7        2039      KUALA LUMPUR          15:51    PULAU SEBANG        18:18
8        2047      KUALA LUMPUR   

In [ ]:
def get_train_schedules(file_map, selected_route, selected_schedule, departure, destination):
    """
    Fetches valid train schedules between departure and destination stations
    based on route and schedule selection using parquet files mapped by file_map.

    Parameters:
        file_map (dict): Dictionary mapping (route, schedule) keys to list of two parquet file paths.
        selected_route (str): Selected route name.
        selected_schedule (str): Selected schedule name.
        departure (str): Departure station name.
        destination (str): Destination station name.

    Returns:
        pd.DataFrame: DataFrame with columns ['Service_ID', 'Departure_Station', 
                    'Departure_Time', 'Arrival_Station', 'Arrival_Time'] or empty DataFrame.
    """
    
    # Key to access the file map
    key = (selected_route, selected_schedule)
    
    # Retrieve the two files from file_map
    if key not in file_map:
        st.error(f"No files found for route '{selected_route}' and schedule '{selected_schedule}'.")
        return pd.DataFrame()

    files = file_map[key]
    if len(files) != 2:
        st.warning("Exactly two files are expected for each route-schedule combination.")
        return pd.DataFrame()

    try:
        df1 = pd.read_parquet(files[0])
        df2 = pd.read_parquet(files[1])
    except Exception as e:
        st.error(f"Error reading parquet files: {e}")
        return pd.DataFrame()

    # Debug info (optional, can be removed in production)
    st.debug(f"Loaded df1 shape: {df1.shape}, df2 shape: {df2.shape}")

    # Helper function to find station index
    def find_station_index(df, station):
        if 'STATION' not in df.columns:
            return float('inf')
        matching_rows = df.index[df['STATION'] == station].tolist()
        return matching_rows[0] if matching_rows else float('inf')

    # Find departure station index in both DataFrames
    idx1 = find_station_index(df1, departure)
    idx2 = find_station_index(df2, departure)

    # Choose the DataFrame where departure station appears earlier (or only in one)
    if idx1 == float('inf') and idx2 == float('inf'):
        st.warning(f"Departure station '{departure}' not found in either dataset.")
        return pd.DataFrame()
    elif idx1 <= idx2:
        chosen_df = df1
        st.info(f"Using data from first file because '{departure}' appears earlier (or only there).")
    else:
        chosen_df = df2
        st.info(f"Using data from second file because '{departure}' appears earlier (or only there).")

    # Validate both stations exist in the chosen DataFrame
    if departure not in chosen_df['STATION'].values:
        st.error(f"❌ Departure station '{departure}' not found.")
        return pd.DataFrame()
    if destination not in chosen_df['STATION'].values:
        st.error(f"❌ Destination station '{destination}' not found.")
        return pd.DataFrame()

    valid_services = []

    # Helper to check if time value is non-empty
    def is_non_empty_time(val):
        if pd.isna(val):
            return False
        if isinstance(val, str):
            return val.strip() != ''
        return True  # Assume non-null non-string values are valid

    # Time string to minutes since midnight for validation
    def time_to_minutes(t):
        t_str = str(t).strip()
        if ':' in t_str:
            h, m = t_str.split(':')
        elif len(t_str) <= 3:  # e.g., "615" → "6:15"
            h, m = "0", t_str.zfill(2)
        else:  # e.g., "1026" → "10:26"
            h, m = t_str[:-2], t_str[-2:]
        return int(h) * 60 + int(m)

    # Loop through all service columns (skip 'STATION')
    for col in chosen_df.columns[1:]:
        dep_row = chosen_df[chosen_df['STATION'] == departure]
        dest_row = chosen_df[chosen_df['STATION'] == destination]

        dep_time = dep_row.iloc[0][col]
        dest_time = dest_row.iloc[0][col]

        # Skip if no valid departure or arrival time
        if not is_non_empty_time(dep_time) or not is_non_empty_time(dest_time):
            continue

        # Optional: Validate time order (departure before arrival)
        try:
            dep_minutes = time_to_minutes(dep_time)
            dest_minutes = time_to_minutes(dest_time)
            if dep_minutes >= dest_minutes:
                continue  # Invalid time sequence
        except Exception:
            pass  # If parsing fails, assume it's acceptable

        # Add valid service
        valid_services.append({
            'Service_ID': col,
            'Departure_Station': departure,
            'Departure_Time': str(dep_time).strip(),
            'Arrival_Station': destination,
            'Arrival_Time': str(dest_time).strip()
        })

    # Final result
    if valid_services:
        schedules = pd.DataFrame(valid_services)
        st.success(f"✅ Found {len(schedules)} valid train service(s) from '{departure}' to '{destination}'.")
        return schedules
    else:
        st.warning(f"🚫 No valid train services found from '{departure}' to '{destination}'. "
                "Check station names or timing data.")
        return pd.DataFrame()

In [6]:
schedules


,Service_ID,Departure_Station,Departure_Time,Arrival_Station,Arrival_Time
0,2007,KUALA LUMPUR,6:16,PULAU SEBANG,9:13
1,2011,KUALA LUMPUR,7:46,PULAU SEBANG,10:13
2,2015,KUALA LUMPUR,8:46,PULAU SEBANG,11:53
3,2019,KUALA LUMPUR,10:26,PULAU SEBANG,12:53
4,2027,KUALA LUMPUR,12:51,PULAU SEBANG,15:18
5,2031,KUALA LUMPUR,13:51,PULAU SEBANG,16:18
6,2035,KUALA LUMPUR,14:51,PULAU SEBANG,17:18
7,2039,KUALA LUMPUR,15:51,PULAU SEBANG,18:18
8,2047,KUALA LUMPUR,17:51,PULAU SEBANG,20:13
9,2049,KUALA LUMPUR,18:06,PULAU SEBANG,20:28
